In [123]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

def get_db_df():
    load_dotenv()
    
    conn = psycopg2.connect(os.getenv('DATABASE_URL'))
    
    # Open a cursor to perform database operations
    cur = conn.cursor()
    
    # Execute a query
    cur.execute("""
    SELECT * FROM united_trades
    where 
    abs(sol_delta) > 0.1
    and abs(token_delta) > 0.01
    order by timestamp
    limit 50000
    """)
    
    # Retrieve query results
    records = cur.fetchall()
    
    # Convert to pandas DataFrame
    df_from_db = pd.DataFrame(records, columns=[desc[0] for desc in cur.description])
    
    # Close communication with the database
    cur.close()
    conn.close()
    
    return df_from_db

df_from_db = get_db_df()

In [124]:
df_from_db

,source,signature,timestamp,mint,token_delta,sol_delta,trader
0,sol,5qWeD3HaQRDYmmMaSzc7ZLtR7T9GfJDFfvRJ3JuUZMAoRp...,2023-12-26 23:42:12,2EgRd5AdhxeBtQ8iAgYQoSBtrbsuLGmDq1wQrPZr7KLs,8.694103e+04,-0.200705,None
1,sol,6LSYVK4dmyKTX9ru22SYzRWKQ2JT4PZ9mnfq5x3MFniCt3...,2023-12-26 23:42:14,7rhhtc2DTTAsPr9P3ZRqUjYZESRj2B29L6qrEXG4VLkh,-1.831851e+06,4.293143,None
2,sol,677uidK1chWa9B8b6VPTEyk6SbBGpwAaB9SZZg234KAw4y...,2023-12-26 23:42:21,GqqjcyCSzoxtGHrfHr33JcUR3xTwNZw9T8KD4pFqStSv,6.064050e+08,-1.000145,None
3,sol,5QTABc6v6TyuJq4CT2TFCS8WRB1q2zGsEMNHiof7qvtFCo...,2023-12-26 23:42:22,93RC484oMK5T9H89rzT5qiAXKHGP9jscXfFfrihNbe57,-1.500000e+02,2.975218,None
4,sol,WQMqcwKwJ7K6LqSZz2ZaXKkRzxV3eCWTd2kjoVhYLuh1UJ...,2023-12-26 23:42:25,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,-2.642058e+03,23.776658,None
...,...,...,...,...,...,...,...
49995,sol,4T6iLCsd7dzXYqtXdD2DfALog21mSL5zVDvVNn4WFWx89V...,2023-12-29 21:51:18,9gwTegFJJErDpWJKjPfLr2g2zrE3nL1v5zpwbtsk3c6P,-8.649436e+02,0.237151,None
49996,sol,3ardbU6Xg7vF49XDi4JT7CYgPSBn7xxWNVBCWje9Apxvg7...,2023-12-29 21:51:18,4jZXkSNgTQKCDb36ECZ6a2aNzcUniGcDeXgTdtM2HxAX,-1.771235e+06,3.212379,None
49997,token,65ATtpaokKCsRheUb4KnV2pkg7DF1fTGp4yNGqZaRx2shg...,2023-12-29 21:51:22,3WoJHGNAcQpjuiT4FqbkqooWkcCMKC5oYDTWjAM9cMmS,-8.498612e+09,1.126034,None
49998,sol,5myF51qh45hPDqLT2cJAQ4pmTx6b3gpGKP8rourASjQYTC...,2023-12-29 21:51:32,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,4.056703e+01,-0.390007,None


In [125]:
df_from_db.describe()

,timestamp,token_delta,sol_delta
count,50000,5.000000e+04,50000.000000
mean,2023-12-28 23:57:46.976639744,-4.199909e+12,0.962761
min,2023-12-26 23:42:12,-7.871866e+16,-5000.083480
25%,2023-12-28 13:58:43.249999872,-8.517645e+03,-1.002832
50%,2023-12-28 23:18:35.500000,-2.591963e-01,0.106032
75%,2023-12-29 11:00:18,5.881324e+03,1.359609
max,2023-12-29 21:51:39,3.839719e+16,25009.120798
std,NaN,5.600269e+14,136.346357


In [126]:
df_from_db.memory_usage(deep=True).sum()

20434294

In [127]:
trades_df = df_from_db.copy()

In [128]:
from datetime import timedelta
TIME_QUANTIZATION = '10min'
QUANT_TIMEDELTA = timedelta(minutes=10)

def add_time_quantization(trades_df):
    trades_df['time_quant'] = trades_df['timestamp'].dt.floor('10min')

add_time_quantization(trades_df)

In [129]:
def remove_rare_tokens(trades_df, min_trades=10, min_quants=3):
    # mints that appears in at least 10 trades
    mints_counts = trades_df.groupby('mint').size()
    mints_with_n_trades = mints_counts[mints_counts >= min_trades].index
    # print(mints_with_10_trades)
    # mints that appears in at least 3 pairs (mint, time_quant)
    keys_counts = trades_df.groupby(['mint', 'time_quant']).size()
    mints_with_n_quants = keys_counts[keys_counts >= min_quants].index.get_level_values(0).unique()
    # print(mints_with_3_pairs)
    # intersection of these mints
    mints_to_keep = mints_with_n_trades.intersection(mints_with_n_quants)
    # filter df by these mints
    trades_df = trades_df[trades_df.mint.isin(mints_to_keep)]
    return trades_df
    
print(len(trades_df))
print(trades_df.mint.nunique())
trades_df = remove_rare_tokens(trades_df)
print(len(trades_df))
print(trades_df.mint.nunique())


50000
1141
45405
231


In [130]:
from typing import Dict
import numpy as np
from dataclasses import dataclass

@dataclass
class PriceTables:
    quants: pd.DatetimeIndex
    sol_prices: Dict[str, np.array]
    united_prices: Dict[str, np.array]

def get_exponential_price_average(df, alpha=0.5) -> float:
    # calculates exponential moving average of price,
    # with sol_delta / token_delta as price,
    # and sol_delta as weight
    prices = abs(df.sol_delta / df.token_delta)
    weights = abs(df.sol_delta)
    # multiply weight exponentially
    exp = (1 - alpha) ** np.arange(len(df))
    # reverse last series
    exp = exp[::-1]
    # multiply weight by exp and make sum equal to 1
    weights = weights * exp
    weights = weights / weights.sum()
    # calc weighted average price
    price = (prices * weights).sum()
    return price

def get_price_tables(trades_df) -> PriceTables:
    min_quant = trades_df.time_quant.min()
    max_quant = trades_df.time_quant.max()
    global_quants = pd.date_range(min_quant, max_quant, freq=TIME_QUANTIZATION)
    quants_len = len(global_quants)
    
    trades_df.sort_values(by=['mint', 'time_quant'], inplace=True)
    sol_prices = {}
    united_prices = {}
    for mint in trades_df.mint.unique():
        sol_prices[mint] = np.zeros(quants_len)
        united_prices[mint] = np.zeros(quants_len)
        mint_df = trades_df[trades_df.mint == mint]
        mint_df.set_index('time_quant', inplace=True)
        last_sol_price = 0.
        last_united_price = 0.
        quants_in_df = mint_df.index.unique()
        for i, quant in enumerate(global_quants):
            if quant in quants_in_df:
                mint_quant_df = mint_df.loc[[quant]]
                last_united_price = get_exponential_price_average(mint_quant_df)
                # get only sol trades which means source is 'sol
                sol_df = mint_quant_df[mint_quant_df['source'] == 'sol'] 
                # if it's non-empty, calc price
                if len(sol_df) > 0:
                    last_sol_price = get_exponential_price_average(sol_df)
            sol_prices[mint][i] = last_sol_price
            united_prices[mint][i] = last_united_price
    return PriceTables(sol_prices=sol_prices, united_prices=united_prices, quants=global_quants)

price_tables = get_price_tables(trades_df)

In [131]:
mint = 'bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1'

up = price_tables.united_prices[mint]
sp = price_tables.sol_prices[mint]
quants = price_tables.quants
# unite them in one df
pdf = pd.DataFrame({
    'quant': quants,
    'united': up, 'sol': sp
})
pdf

,quant,united,sol
0,2023-12-26 23:40:00,0.000000,0.000000
1,2023-12-26 23:50:00,1.099779,1.099779
2,2023-12-27 00:00:00,1.099779,1.099779
3,2023-12-27 00:10:00,1.099779,1.099779
4,2023-12-27 00:20:00,1.099779,1.099779
...,...,...,...
417,2023-12-29 21:10:00,1.098857,1.100309
418,2023-12-29 21:20:00,1.098857,1.100309
419,2023-12-29 21:30:00,0.003433,1.100309
420,2023-12-29 21:40:00,1.099969,1.099969


In [134]:
pdf.describe()

,quant,united,sol
count,422,422.000000,422.000000
mean,2023-12-28 10:45:00.000000512,1.280526,1.285093
min,2023-12-26 23:40:00,0.000000,0.000000
25%,2023-12-27 17:12:30,1.099467,1.099717
50%,2023-12-28 10:45:00,1.099779,1.099779
75%,2023-12-29 04:17:30,1.099845,1.099947
max,2023-12-29 21:50:00,40.710355,40.710355
std,NaN,2.725305,2.724358


In [135]:
interesting_quant = '2023-12-28 19:10:00'
# filter by quant and mint
trades_df[(trades_df.mint == mint) & (trades_df.time_quant == interesting_quant)]


,source,signature,timestamp,mint,token_delta,sol_delta,trader,time_quant
19829,sol,2rTduCbHRiM2EAdkWHbRUDsrgD4ZSiKUfToWEgLr8htW8e...,2023-12-28 19:10:18,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,-5.303587,5.831234,7YmiQpsYeJ8TPc9sFcJbMCTTUYjXuixCEiwyzyML18yz,2023-12-28 19:10:00
20008,sol,3u2BRaWmTvLVh3vUh3x4hUu4QxVggFmKRp6WXBk84vaZ2P...,2023-12-28 19:17:43,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.507837,-23.202044,4W55gPe3PD4DaJxPVpLEbnNV5yRCQieFVABQVbeBvsPg,2023-12-28 19:10:00


In [119]:
interesting_quant = '2023-12-28 21:00:00'
# filter by quant and mint
trades_df[(trades_df.mint == mint) & (trades_df.time_quant == interesting_quant)]


,source,signature,timestamp,mint,token_delta,sol_delta,trader,time_quant
20683,token,568wJrnDbEjM8n4d3UWJKX2EddsZYRBqfB9F1tackTVXmb...,2023-12-28 21:06:47,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,0.392477,-0.42996,5M5yosrLEUQRnusnnQmfBq2c7btQ6AXvMxx43EUsfaXW,2023-12-28 21:00:00
